## Feature Engineering

In [ ]:
def tokenize(document):
    document = document.split('|')
    document = [word.lower() for word in document if word not in stopwords.words('english')]
    return document
df_train['KEYWORDS']= df_train['KEYWORDS'].apply(lambda x : tokenize(x))
df_test['KEYWORDS']= df_train['KEYWORDS'].apply(lambda x : tokenize(x))

In [ ]:
def reg(url):
    website = url.split('/')[2]
    name = re.search(r"[^www.]+[^.com]", website)
    return name.group()
wesbite_name = df['URL'].apply(reg)

In [ ]:
## website name extraction
def reg(url):
    website = url.split('/')[2]
    name = re.search(r"[^www.]+[^.com]", website)
    return name.group()
df['extracted'] = df['URL'].apply(reg)

## website path extraction
def re2(url):
    return re.split("^(?:https?:)?(?:\/\/)?(?:[^@\n]+@)?(?:www\.)?([^:\/\n]+)",url)
df['extracted2'] = df['URL'].apply(re2)

def ret2(e2):
    return e2[2]
df['extracted2']= df['extracted2'].apply(ret2)

def sl(e2):
    return e2[1:]
df['extracted2'] = df['extracted2'].apply(sl)

def sp(e2):
    ne2= e2.replace('/', '-').split('-')
    return ne2
df['extracted2'] = df['extracted2'].apply(sp)

def remo(e2):
    stopwords=["home","articles","article","feature","features","topic","topics","news","slideshow","slideshows","souce","sources"]
    for i in e2:
        if i in stopwords:
            e2.remove(i)
    return e2
df['extracted2'] = df['extracted2'].apply(remo)

def tot(e2):
    s=""
    for e in e2:
        s=s+e+" "
    return s
df['extracted2'] = df['extracted2'].apply(tot)

from sklearn import preprocessing


## Fasstext Embeddings

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import VotingClassifier,StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import preprocessing

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from gensim.models import Word2Vec,KeyedVectors

In [3]:
import nltk
from nltk.corpus import stopwords

In [4]:
df_train  = pd.read_csv('train_modified.csv')
df_test  = pd.read_csv('test_modified.csv')

In [5]:
import re

def url_web(url):
    website = url.split('/')[2]
    name = re.search(r"[^www.]+[^.com]", website)
    return name.group()

df_train['WEBSITE'] = df_train['URL'].apply(lambda x: url_web(x))
df_test['WEBSITE']= df_test['URL'].apply(lambda x:url_web(x))

In [6]:
url_keyword = pd.read_csv('Url_keyword.csv')

In [20]:
url_test_keyword = pd.read_csv('Url_keyword_test.csv')

In [18]:
url_keyword.extracted2.isna().sum()

7646

In [33]:
url_keyword = url_keyword.replace(np.nan, '', regex=True)

In [39]:
url_test_keyword = url_test_keyword.replace(np.nan, '', regex=True)

In [48]:
temp_df  = pd.read_csv('train_modified.csv')

In [49]:
df_train['KEYWORDS'] = temp_df['KEYWORDS'] + "  " + url_keyword['extracted2']

In [50]:
temp_df  = pd.read_csv('test_modified.csv')

In [51]:
df_test['KEYWORDS'] = temp_df['KEYWORDS'] + "  " + url_test_keyword['extracted2'] 

In [52]:
df_train['KEYWORDS'][113935]

'genetic large autoimmune health false abortion dermatology ophthalmology heart failure technology trauma surgery flu clinical lung abscess analgesia oncology lung rheumatoid cardiovascular anemia hereditary cardiology rheumatology abscess arthritis anesthesia lung diseases medicine dermatitis biliary tract congenital oral neurology molecular physicians liver asthma ruptured angioedema diagnosis gastroenterology psychiatry urology  '

In [53]:
def duplicate_remove(text):
    text = text.split()
    text = list(set(text))
    return text
df_train['KEYWORDS'] = df_train['KEYWORDS'].apply(lambda x : duplicate_remove(x))
df_test['KEYWORDS'] = df_test['KEYWORDS'].apply(lambda x : duplicate_remove(x))


In [54]:
df_train['KEYWORDS']

0         [psychiatry, cardiology, prostate, surgery, ch...
1         [lymphoid, small, bone, cells, large, psychiat...
2         [psychiatry, cardiology, prostate, surgery, ch...
3         [false, consult, maintenance, medcalcs, physic...
4         [pathology, small, reproductive, neutrophils, ...
                                ...                        
113931    [male, psychiatry, cardiology, surgery, health...
113932    [psychiatry, cardiology, features, surgery, he...
113933    [pathology, small, dentistry, myocardial, larg...
113934    [mail, practitioners, ee, psoriasis, cardiolog...
113935    [abscess, large, analgesia, abortion, psychiat...
Name: KEYWORDS, Length: 113936, dtype: object

In [57]:
from gensim.models import FastText
ft = FastText(vector_size=128, window=1, min_count=2)
ft.build_vocab(df_train.KEYWORDS)

In [59]:
len(ft.wv.index_to_key)

8606

In [62]:
from tqdm import tqdm

In [63]:
def document_vector(doc):
    doc = [word for word in doc if word in ft.wv.index_to_key]
    return np.mean(ft.wv[doc], axis=0)

In [64]:
W2V_train = []
for doc in tqdm(df_train['KEYWORDS'].values):
    W2V_train.append(document_vector(doc))

100%|████████████████████████████████████████████████████████████████████████| 113936/113936 [00:21<00:00, 5376.82it/s]


In [65]:
W2V_test = []
for doc in tqdm(df_test['KEYWORDS'].values):
    W2V_test.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████| 28493/28493 [00:05<00:00, 5293.62it/s]


In [67]:
df_vector = pd.DataFrame(W2V_train)

In [72]:
df_test_vector = pd.DataFrame(W2V_test)

In [73]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
train_vector_df_scaled = std_scaler.fit_transform(df_vector.to_numpy())
test_vector_df_scaled = std_scaler.transform(df_test_vector.to_numpy())

In [71]:
df_train['USERZIPCODE'].value_counts()

63169.0        2116
11226.0        1752
22202.0        1259
10001.0        1190
90060.0        1058
               ... 
56748.0           1
77021.0           1
74880.0           1
93630.0           1
760865488.0       1
Name: USERZIPCODE, Length: 11278, dtype: int64

In [74]:
train_df = df_train[['DEVICETYPE','PLATFORM_ID','PLATFORMTYPE','WEBSITE']]
test_df = df_test[['DEVICETYPE','PLATFORM_ID','PLATFORMTYPE','WEBSITE']]

In [75]:
train_df

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE
0,Desktop,2,Online Medical Journal,cancertherapyadvisor
1,Desktop,2,Online Medical Journal,cancertherapyadvisor
2,Desktop,2,Online Medical Journal,cancertherapyadvisor
3,Desktop,3,Online Medical Journal,globalrph
4,Mobile,7,Online Medical Journal,cureus
...,...,...,...,...
113931,Desktop,2,Online Medical Journal,clinicaladvisor
113932,Desktop,2,Online Medical Journal,clinicaladvisor
113933,Desktop,7,Online Medical Journal,cureus
113934,Mobile,2,Online Medical Journal,dermatologyadvisor


In [76]:
from sklearn import preprocessing 
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()
le4 = preprocessing.LabelEncoder()

In [ ]:
train_df['DEVICETYPE'] = le3.fit_transform(train_df['DEVICETYPE'])
train_df['PLATFORMTYPE'] = le2.fit_transform(train_df['PLATFORMTYPE'])
train_df['WEBSITE'] = le4.fit_transform(train_df['WEBSITE'])

In [ ]:
test_df['DEVICETYPE'] = le3.transform(test_df['DEVICETYPE'])
test_df['PLATFORMTYPE'] = le2.transform(test_df['PLATFORMTYPE'])
test_df['WEBSITE'] = le4.transform(test_df['WEBSITE'])

In [83]:
train_df_scaled = pd.concat([train_df,pd.DataFrame(train_vector_df_scaled.tolist())],axis=1)
train_df_scaled

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,0,1,2,3,4,5,...,118,119,120,121,122,123,124,125,126,127
0,0,2,3,2,-1.057447,-0.403736,1.318727,0.378889,1.874817,0.850152,...,1.546720,0.698541,-1.594306,-1.492440,0.219206,1.250449,0.032465,0.610927,-0.794986,-0.826591
1,0,2,3,2,-0.675701,0.258334,-0.762873,-0.282886,1.193061,1.210772,...,1.171981,0.036620,1.410534,-0.556974,0.453055,-1.811381,-1.519520,-0.462044,-0.292856,-0.192163
2,0,2,3,2,-1.057447,-0.403736,1.318727,0.378889,1.874817,0.850152,...,1.546720,0.698541,-1.594306,-1.492440,0.219206,1.250449,0.032465,0.610927,-0.794986,-0.826591
3,0,3,3,13,-1.934873,2.823090,-0.996072,-0.671552,-0.602340,-0.930940,...,0.990387,-1.760160,-1.450108,2.106703,1.452325,1.806776,1.775664,3.026083,0.062629,-0.956066
4,1,7,3,6,-0.191290,0.950974,-0.189448,0.039269,-0.836980,0.136970,...,-0.185974,-0.727367,-0.424088,-0.687747,-0.754157,0.646184,-0.663608,-0.995548,0.315013,-1.196848
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113931,0,2,3,3,-1.502910,-0.972426,0.145562,0.789978,0.638443,1.741063,...,0.652251,0.374173,0.844582,1.065414,0.883206,0.908350,0.137322,0.029551,-0.327286,-0.818684
113932,0,2,3,3,-2.167361,-1.798454,1.408415,1.619750,0.686749,-0.377136,...,-0.233013,2.681397,0.414248,-0.541311,2.227016,1.054836,-0.164982,0.131731,-1.717068,-0.298195
113933,0,7,3,6,0.187964,0.876819,-0.889709,-0.758830,-0.214027,0.265852,...,-0.605874,-0.088441,-0.132815,0.254190,-0.782063,-0.137567,0.234847,-0.640986,0.047425,-1.423571
113934,1,2,3,7,2.170612,-0.685427,4.936308,0.216777,0.740000,0.245893,...,0.548233,-1.572463,-0.863311,-2.564533,-1.547825,1.602381,2.016485,0.477434,2.757289,3.022198


In [84]:
test_df_scaled = pd.concat([test_df,pd.DataFrame(test_vector_df_scaled.tolist())],axis=1)
test_df_scaled

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,0,1,2,3,4,5,...,118,119,120,121,122,123,124,125,126,127
0,0,2,3,3,-0.601669,-0.806271,0.529971,0.717364,-0.865731,-0.580379,...,0.928971,0.160925,0.744432,0.812480,-0.013414,-0.687134,0.853638,-1.259637,-0.232440,-0.379216
1,1,2,3,22,0.413631,-0.765350,1.567767,-2.075026,2.325544,-1.258707,...,1.215812,1.175872,-0.178750,-1.918315,-1.007472,0.133548,-1.791655,-1.078461,0.624321,-1.092045
2,0,2,3,25,0.209400,-0.840555,0.183702,0.884861,-1.831957,0.154359,...,-0.683028,0.799729,-0.320745,0.619316,0.907153,1.543077,1.931017,-0.854613,-0.350372,-0.560973
3,0,7,3,6,0.813153,1.547024,-0.509471,-0.186557,-0.158283,-0.056781,...,0.405432,-0.539547,-0.938188,-0.638081,-0.801283,0.489561,0.398383,-1.005311,0.037701,-1.480856
4,0,2,3,30,1.700441,-0.761901,0.790399,-2.163829,-0.969484,1.001109,...,-0.868386,0.793355,-1.576947,-0.876424,-0.731949,-0.269267,0.627223,1.051835,0.962608,0.740791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28488,0,2,3,30,0.666005,-0.376829,1.730511,0.349635,-0.235752,0.677732,...,-0.105713,-0.729631,-0.143042,0.254338,1.036926,0.612329,-0.834829,-1.054059,1.271808,-0.075827
28489,0,7,3,6,-0.478236,0.357445,-0.516194,-0.677600,0.576401,-0.521194,...,0.538333,-0.023285,-1.289090,0.161483,-0.381501,-1.015584,1.019760,0.614514,-0.016737,-1.137491
28490,0,7,3,6,-0.222371,0.221816,-0.633484,-0.322181,1.015192,-0.330940,...,-0.094223,-0.284707,-1.310844,0.096216,-0.572418,-0.874780,0.548458,0.128006,-0.475749,-1.540477
28491,0,8,3,28,3.122868,-1.099979,1.565310,1.393845,-1.853954,3.235036,...,-1.942933,-1.808739,-3.657810,-2.595916,-0.317350,0.186879,1.574490,1.008268,0.818969,-0.435895


In [85]:
train_df_scaled.columns = train_df_scaled.columns.astype('str')
test_df_scaled.columns = test_df_scaled.columns.astype('str')

In [86]:
y = df_train['IS_HCP']

In [88]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy=1.0)
X_sm,y_sm = smote.fit_resample(train_df_scaled,y)

y_sm.value_counts()

0    79756
1    79756
Name: IS_HCP, dtype: int64

In [210]:
rf_ft = RandomForestClassifier()
rf_ft.fit(X_sm,y_sm)
rf_pred = rf_ft.predict(test_df_scaled)

In [155]:
test_test_df.IS_HCP = rf_pred
test_test_df.to_csv('seventythree_test_csv.csv',index = False)

In [211]:
test_test_df.IS_HCP = rf_pred
test_test_df.to_csv('seventynine_test_csv.csv',index = False)

In [201]:
from xgboost import XGBClassifier

xgb_ft = XGBClassifier()
xgb_ft.fit(X_sm,y_sm)
xgb_pred = xgb_ft.predict(test_df_scaled)
test_test_df.IS_HCP = xgb_pred
test_test_df.to_csv('seventyfive_test_csv.csv',index = False)

In [ ]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

space={ 'objective':'binary:logistic',
        'learning_rate':hp.uniform('learning_rate',0.1,0.4),
        'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 0,9),
        'reg_alpha' : hp.quniform('reg_alpha', 0,1,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_sm, y_sm, test_size=0.20, random_state=42)
def objective(space):
    clf=XGBClassifier(objective = space['objective'],learning_rate = space['learning_rate'],
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train_2, y_train_2), ( X_test_2, y_test_2)]
    
    clf.fit(X_train_2, y_train_2,
             eval_set=evaluation,eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    
    pred = clf.predict(X_test_2)
    accuracy = accuracy_score(y_test_2, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)
best_hyperparams['max_depth'] = int(best_hyperparams['max_depth'])
print(best_hyperparams)

In [204]:
xgb_ft_2 = XGBClassifier()
xgb_ft_2.set_params(**best_hyperparams)
xgb_ft_2.fit(X_sm,y_sm)
xgb_ft_pred = xgb_ft_2.predict(test_df_scaled)

In [205]:
test_test_df.IS_HCP = xgb_ft_pred
test_test_df.to_csv('seventysix_test_csv.csv',index = False)

In [206]:
dt_ft = DecisionTreeClassifier()
dt_ft.fit(X_sm,y_sm)
dt_pred = dt_ft.predict(test_df_scaled)
test_test_df.IS_HCP = dt_pred
test_test_df.to_csv('seventyseven_test_csv.csv',index = False)

In [208]:
knn_ft = KNeighborsClassifier()
knn_ft.fit(X_sm,y_sm)
knn_pred = knn_ft.predict(test_df_scaled)
test_test_df.IS_HCP = knn_pred
test_test_df.to_csv('seventyeight_test_csv.csv',index = False)

In [216]:
vc_ft_soft_2 = VotingClassifier([('clf1',rf_ft),('clf2',xgb_ft)],voting = 'soft')
vc_ft_soft_2.fit(X_sm,y_sm)
vc_pred_2 = vc_ft_soft_2.predict(test_df_scaled)

In [215]:
test_test_df.IS_HCP = vc_pred_2
test_test_df.to_csv('seventynine_test_csv.csv',index = False)

In [217]:
from sklearn.svm import SVC
svm = SVC(kernel = 'rbf', random_state = 0)
svm.fit(X_sm, y_sm)
svm_pred = svm.predict(test_df_scaled)
test_test_df.IS_HCP = svm_pred
test_test_df.to_csv('seventynine_test_csv.csv',index = False)

In [219]:
estimators = [('clf1',xgb_ft),('clf2',rf_ft)]
stack_model = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())
stack_model.fit(X_sm,y_sm)
stack_pred = stack_model.predict(test_df_scaled)

In [221]:
vc_ft_soft_3 = VotingClassifier([('clf1',vc_ft_soft_2),('clf2',stack_model)],voting = 'soft')
vc_ft_soft_3.fit(X_sm,y_sm)
vc_pred_3 = vc_ft_soft_3.predict(test_df_scaled)

In [224]:
estimators = [('clf1',vc_ft_soft_3),('clf2',vc_ft_soft_2)]
stack_model = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())
stack_model.fit(X_sm,y_sm)
stack_pred = stack_model.predict(test_df_scaled)

In [225]:
test_test_df.IS_HCP = stack_pred
test_test_df.to_csv('eightytwo_test_csv.csv',index = False)

In [565]:
X_sm_4['IS_HCP'] = y_sm_4

In [567]:
X_sm_4.to_csv('training_dataframe_final.csv',index=False)

## SBert embeddings 384 dim

In [227]:
def key2(text):
    sentence = " ".join(text)
    return sentence

df_train['SENTENCE'] = df_train['KEYWORDS'].apply(key2)
df_test['SENTENCE'] = df_test['KEYWORDS'].apply(key2)

In [560]:
X_sm_2

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,CITY,0,1,2,3,4,...,374,375,376,377,378,379,380,381,382,383
0,0,2,3,2,3275,0.062621,0.030104,-0.049696,-0.002872,-0.077366,...,-0.005600,0.025201,0.041644,-0.008444,-0.049387,-0.011292,0.105604,-0.026650,0.009321,0.075258
1,0,2,3,2,132,0.052006,0.048003,0.000151,-0.057583,-0.041250,...,0.010245,0.078027,0.041478,0.027773,0.064026,-0.029505,0.008016,0.048607,0.014628,0.026060
2,0,2,3,2,2781,0.062621,0.030104,-0.049696,-0.002872,-0.077366,...,-0.005600,0.025201,0.041644,-0.008444,-0.049387,-0.011292,0.105604,-0.026650,0.009321,0.075258
3,0,3,3,13,2838,-0.033458,-0.012506,-0.067504,-0.047726,-0.135716,...,-0.086089,0.069374,0.112561,0.030620,-0.005797,-0.048223,-0.018869,-0.102295,0.032811,0.014253
4,1,7,3,6,1820,-0.039615,-0.005113,-0.023556,-0.009341,-0.089699,...,-0.000015,0.088376,0.020650,0.020895,0.016181,0.038231,-0.041902,-0.014496,0.114087,0.040825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159507,0,5,3,11,3022,-0.057195,0.020465,-0.016369,0.003152,0.021515,...,0.009517,0.040098,0.016412,-0.019550,0.013423,-0.016449,0.099579,-0.053272,0.080581,0.105289
159508,0,7,3,6,3200,0.002501,0.043548,0.008923,-0.030592,-0.081578,...,0.020241,0.099163,0.049956,0.011153,-0.038459,0.006842,0.046380,0.005827,0.075120,0.015936
159509,0,10,3,15,477,-0.023614,0.080377,0.003659,-0.013189,-0.103376,...,-0.053719,0.074557,0.128828,-0.008775,-0.031185,0.075569,0.080364,-0.018243,0.119820,0.080047
159510,0,7,3,6,502,-0.005699,0.052850,-0.029038,-0.022706,-0.065682,...,-0.005977,0.102965,0.069319,0.033486,0.018661,-0.010073,0.091102,-0.008341,0.110261,-0.003359


In [229]:
df_train['SENTENCE']

0         psychiatry cardiology prostate surgery chronic...
1         lymphoid small bone cells large psychiatry gla...
2         psychiatry cardiology prostate surgery chronic...
3         false consult maintenance medcalcs physicians ...
4         pathology small reproductive neutrophils denti...
                                ...                        
113931    male psychiatry cardiology surgery health rela...
113932    psychiatry cardiology features surgery health ...
113933    pathology small dentistry myocardial large phy...
113934    mail practitioners ee psoriasis cardiology mul...
113935    abscess large analgesia abortion psychiatry ca...
Name: SENTENCE, Length: 113936, dtype: object

In [230]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

100%|█████████████████████████████████████████████████████████████████████████| 113936/113936 [15:30<00:00, 122.39it/s]


In [232]:
embeddings_test = []
for i in tqdm(df_test['SENTENCE']):
    embeddings_test.append(model.encode(i))

100%|███████████████████████████████████████████████████████████████████████████| 28493/28493 [03:53<00:00, 122.09it/s]


In [233]:
df_bert_embeddings_train = pd.DataFrame(embeddings_train)
df_bert_embeddings_test = pd.DataFrame(embeddings_test)

In [234]:
df_bert_embeddings_train

,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.062621,0.030104,-0.049696,-0.002872,-0.077366,-0.075697,0.011184,0.063107,-0.022533,-0.014627,...,-0.005600,0.025201,0.041644,-0.008444,-0.049387,-0.011292,0.105604,-0.026650,0.009321,0.075258
1,0.052006,0.048003,0.000151,-0.057583,-0.041250,-0.041908,0.052107,0.075072,-0.063839,-0.003724,...,0.010245,0.078027,0.041478,0.027773,0.064026,-0.029505,0.008016,0.048607,0.014628,0.026060
2,0.062621,0.030104,-0.049696,-0.002872,-0.077366,-0.075697,0.011184,0.063107,-0.022533,-0.014627,...,-0.005600,0.025201,0.041644,-0.008444,-0.049387,-0.011292,0.105604,-0.026650,0.009321,0.075258
3,-0.033458,-0.012506,-0.067504,-0.047726,-0.135716,-0.072514,-0.004371,0.128291,0.048798,0.025764,...,-0.086089,0.069374,0.112561,0.030620,-0.005797,-0.048223,-0.018869,-0.102295,0.032811,0.014253
4,-0.039615,-0.005113,-0.023556,-0.009341,-0.089699,0.026324,-0.037572,0.117681,0.006736,0.037044,...,-0.000015,0.088376,0.020650,0.020895,0.016181,0.038231,-0.041902,-0.014496,0.114087,0.040825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113931,0.010681,0.081000,-0.066219,0.003094,-0.111840,-0.052544,-0.004690,0.012551,-0.036467,-0.055640,...,-0.033433,0.058592,0.054049,0.052852,-0.012197,-0.059723,0.042657,-0.026619,0.041272,0.014037
113932,0.008793,0.039040,-0.004619,-0.010275,-0.085800,-0.061926,-0.009247,0.048734,-0.020262,-0.023271,...,-0.072673,0.095038,0.007995,0.045544,0.001883,-0.017897,0.073166,-0.012864,0.019074,0.058617
113933,-0.005699,0.052850,-0.029038,-0.022706,-0.065682,-0.025156,-0.048542,0.111578,0.009178,0.023917,...,-0.005977,0.102965,0.069319,0.033486,0.018661,-0.010073,0.091102,-0.008341,0.110261,-0.003359
113934,0.003200,0.047974,0.016732,0.045390,0.001444,0.030114,0.075824,0.096617,-0.037371,0.004306,...,-0.106074,0.104969,0.089438,-0.008973,0.005175,-0.060500,0.076859,-0.069754,-0.000290,0.091081


In [239]:
train_df_cat = train_df.iloc[:,:4]

In [241]:
test_df_cat = test_df.iloc[:,:4]

In [237]:
train_df_bert = pd.concat([train_df_cat,df_bert_embeddings_train],axis=1)

In [242]:
test_df_bert = pd.concat([test_df_cat,df_bert_embeddings_test],axis=1)

In [245]:
train_df_bert.columns = train_df_bert.columns.astype('str')
test_df_bert.columns = test_df_bert.columns.astype('str')

In [456]:
smote = SMOTE(sampling_strategy=1.0)
X_sm_2,y_sm_2 = smote.fit_resample(train_df_bert,y)

y_sm_2.value_counts()

0    79756
1    79756
Name: IS_HCP, dtype: int64

In [247]:
vc_ft_soft_3.fit(X_sm_2,y_sm_2)
vc_pred_3 = vc_ft_soft_3.predict(test_df_bert)
test_test_df.IS_HCP = vc_pred_3
test_test_df.to_csv('eightythree_test_csv.csv',index = False)

In [249]:
xgb_ft_3 = XGBClassifier()
xgb_ft_3.fit(X_sm_2,y_sm_2)
xgb_ft_pred = xgb_ft_3.predict(test_df_bert)
test_test_df.IS_HCP = xgb_ft_pred
test_test_df.to_csv('eightyfour_test_csv.csv',index = False)

In [ ]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

space={ 'objective':'binary:logistic',
        'learning_rate':hp.uniform('learning_rate',0.1,0.4),
        'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 0,9),
        'reg_alpha' : hp.quniform('reg_alpha', 0,1,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_sm_2, y_sm_2, test_size=0.20, random_state=42)
def objective(space):
    clf=XGBClassifier(objective = space['objective'],learning_rate = space['learning_rate'],
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train_2, y_train_2), ( X_test_2, y_test_2)]
    
    clf.fit(X_train_2, y_train_2,
             eval_set=evaluation,eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    
    pred = clf.predict(X_test_2)
    accuracy = accuracy_score(y_test_2, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

trials = Trials()

best_hyperparams_2 = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)
best_hyperparams_2['max_depth'] = int(best_hyperparams['max_depth'])
print(best_hyperparams_2)

In [252]:
xgb_ft_4 = XGBClassifier()
xgb_ft_4.set_params(**best_hyperparams_2)
xgb_ft_4.fit(X_sm_2,y_sm_2)
xgb_ft_pred = xgb_ft_4.predict(test_df_bert)
test_test_df.IS_HCP = xgb_ft_pred
test_test_df.to_csv('eightyfive_test_csv.csv',index = False)

In [288]:
test_df_bert

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,0,1,2,3,4,5,...,374,375,376,377,378,379,380,381,382,383
0,0,2,3,3,0.010489,0.016602,-0.006979,0.035332,-0.070729,-0.018661,...,-0.047317,0.048213,-0.008160,0.058611,0.008078,-0.007004,0.062972,-0.042054,-0.022759,0.093909
1,1,2,3,22,0.094528,-0.016281,0.024730,0.011690,-0.147179,-0.118570,...,-0.127606,0.051345,0.091650,0.006316,-0.051278,-0.027012,0.098344,0.055415,0.043057,0.024547
2,0,2,3,25,0.027893,-0.006295,-0.029042,0.112363,-0.010424,0.023772,...,0.025637,0.066882,-0.010851,-0.016770,-0.045228,-0.069519,0.141032,-0.021204,0.025619,0.023021
3,0,7,3,6,-0.035346,0.013211,-0.005575,-0.004231,-0.108935,-0.039370,...,-0.016838,0.067102,0.049788,-0.007241,0.010638,0.011966,-0.006030,-0.009339,0.087207,0.066948
4,0,2,3,30,-0.025487,0.062336,0.020333,0.031348,-0.049694,-0.082555,...,-0.001440,0.039353,0.012410,-0.047881,0.078327,-0.007633,0.131235,-0.102498,0.019462,0.089912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28488,0,2,3,30,-0.020699,0.025019,0.058334,0.017866,-0.046980,-0.007158,...,-0.016635,-0.013652,0.038399,-0.010011,0.012348,-0.025183,0.052855,0.029544,0.004742,0.017152
28489,0,7,3,6,-0.014081,0.056701,-0.037410,-0.009524,-0.119517,-0.026023,...,-0.022639,0.161927,0.102655,-0.042218,-0.011919,0.010554,0.048302,-0.044039,0.071084,0.085600
28490,0,7,3,6,-0.006124,0.073382,-0.020083,-0.014053,-0.103585,-0.047098,...,-0.014792,0.158156,0.115152,-0.046444,-0.021150,0.026983,0.062074,-0.054671,0.083501,0.051499
28491,0,8,3,28,-0.042832,0.055927,-0.019467,-0.002870,-0.027572,-0.106546,...,-0.028339,-0.028347,0.014240,0.024539,-0.021474,0.082236,0.028688,0.033081,-0.021743,0.071638


In [559]:
df_train.TAXONOMY.value_counts()

2084P0800X    7930
2084N0400X    6621
207Q00000X    3997
207R00000X    3190
208000000X     837
              ... 
163WC0400X       1
246QM0706X       1
1835N1003X       1
207RI0008X       1
207XX0801X       1
Name: TAXONOMY, Length: 207, dtype: int64

In [290]:
std_scaler_2 = StandardScaler()
train_vector_df_scaled_2 = std_scaler_2.fit_transform(X_sm_2.iloc[:,4:].to_numpy())
test_vector_df_scaled_2 = std_scaler_2.transform(test_df_bert.iloc[:,4:].to_numpy())
train_df_bert_scaled = pd.concat([X_sm_2.iloc[:,:4],pd.DataFrame(train_vector_df_scaled_2.tolist())],axis=1)
test_df_bert_scaled = pd.concat([test_df_cat,pd.DataFrame(test_vector_df_scaled_2.tolist())],axis=1)
train_df_bert_scaled.columns = train_df_bert_scaled.columns.astype('str')
test_df_bert_scaled.columns = test_df_bert_scaled.columns.astype('str')

In [292]:
rf_ft = RandomForestClassifier()
rf_ft.fit(train_df_bert_scaled,y_sm_2)
rf_pred = rf_ft.predict(test_df_bert_scaled)
test_test_df.IS_HCP = rf_pred
test_test_df.to_csv('ninetyfour_test_csv.csv',index = False)

In [293]:
rf_ft = RandomForestClassifier()
rf_ft.fit(X_sm_2,y_sm_2)
rf_pred = rf_ft.predict(test_df_bert)
test_test_df.IS_HCP = rf_pred
test_test_df.to_csv('eightysix_a_test_csv.csv',index = False)

In [ ]:
model_CBC = ctb.CatBoostClassifier(iterations = 2000)
model_CBC.fit(X_sm_2, y_sm_2)

In [295]:
cat_pred = model_CBC.predict(test_df_bert)
test_test_df.IS_HCP = cat_pred 
test_test_df.to_csv('eightysix_b_test_csv.csv',index = False)

In [259]:
rf_ft_2 = RandomForestClassifier()
rf_ft_2.max_features = 'auto'
rf_ft_2.fit(X_sm_2,y_sm_2)
rf_pred = rf_ft_2.predict(test_df_bert)
test_test_df.IS_HCP = rf_pred
test_test_df.to_csv('eightynine_test_csv.csv',index = False)

C:\Users\USER\AppData\Roaming\Python\Python39\site-packages\sklearn\ensemble\_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [257]:
vc_ft_soft_3 = VotingClassifier([('clf1',rf_ft),('clf2',xgb_ft_4)],voting = 'soft')
vc_ft_soft_3.fit(X_sm_2,y_sm_2)
vc_pred_3 = vc_ft_soft_3.predict(test_df_bert)
test_test_df.IS_HCP = vc_pred_3
test_test_df.to_csv('eightyseven_test_csv.csv',index = False)

In [258]:
dt_ft = DecisionTreeClassifier()
dt_ft.fit(X_sm_2,y_sm_2)
dt_pred = dt_ft.predict(test_df_bert)
test_test_df.IS_HCP = dt_pred
test_test_df.to_csv('eightyeight_test_csv.csv',index = False)

In [260]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier()
lgbm.fit(X_sm_2, y_sm_2)
lgbm_pred = lgbm.predict(test_df_bert)
test_test_df.IS_HCP = lgbm_pred
test_test_df.to_csv('ninety_test_csv.csv',index = False)

In [261]:
estimators = [('clf1',rf_ft),('clf2',vc_ft_soft_3)]
stack_model_2 = StackingClassifier(estimators = estimators, final_estimator = LogisticRegression())
stack_model_2.fit(X_sm_2,y_sm_2)
stack_pred = stack_model_2.predict(test_df_bert)
test_test_df.IS_HCP = stack_pred
test_test_df.to_csv('ninetyone_test_csv.csv',index = False)

In [368]:
df_train['NEWCITY']

0             Portland
1            Arlington
2          New Meadows
3                 None
4              Houston
              ...     
113931    Philadelphia
113932        Van Nuys
113933            None
113934      Wilmington
113935    White Plains
Name: NEWCITY, Length: 113936, dtype: object

In [370]:
train_df_bert['CITY'] = df_train['NEWCITY']

In [372]:
train_df_bert.drop(columns = ['CITY'],inplace = True)

In [373]:
train_df_bert

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,0,1,2,3,4,5,...,374,375,376,377,378,379,380,381,382,383
0,0,2,3,2,0.062621,0.030104,-0.049696,-0.002872,-0.077366,-0.075697,...,-0.005600,0.025201,0.041644,-0.008444,-0.049387,-0.011292,0.105604,-0.026650,0.009321,0.075258
1,0,2,3,2,0.052006,0.048003,0.000151,-0.057583,-0.041250,-0.041908,...,0.010245,0.078027,0.041478,0.027773,0.064026,-0.029505,0.008016,0.048607,0.014628,0.026060
2,0,2,3,2,0.062621,0.030104,-0.049696,-0.002872,-0.077366,-0.075697,...,-0.005600,0.025201,0.041644,-0.008444,-0.049387,-0.011292,0.105604,-0.026650,0.009321,0.075258
3,0,3,3,13,-0.033458,-0.012506,-0.067504,-0.047726,-0.135716,-0.072514,...,-0.086089,0.069374,0.112561,0.030620,-0.005797,-0.048223,-0.018869,-0.102295,0.032811,0.014253
4,1,7,3,6,-0.039615,-0.005113,-0.023556,-0.009341,-0.089699,0.026324,...,-0.000015,0.088376,0.020650,0.020895,0.016181,0.038231,-0.041902,-0.014496,0.114087,0.040825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113931,0,2,3,3,0.010681,0.081000,-0.066219,0.003094,-0.111840,-0.052544,...,-0.033433,0.058592,0.054049,0.052852,-0.012197,-0.059723,0.042657,-0.026619,0.041272,0.014037
113932,0,2,3,3,0.008793,0.039040,-0.004619,-0.010275,-0.085800,-0.061926,...,-0.072673,0.095038,0.007995,0.045544,0.001883,-0.017897,0.073166,-0.012864,0.019074,0.058617
113933,0,7,3,6,-0.005699,0.052850,-0.029038,-0.022706,-0.065682,-0.025156,...,-0.005977,0.102965,0.069319,0.033486,0.018661,-0.010073,0.091102,-0.008341,0.110261,-0.003359
113934,1,2,3,7,0.003200,0.047974,0.016732,0.045390,0.001444,0.030114,...,-0.106074,0.104969,0.089438,-0.008973,0.005175,-0.060500,0.076859,-0.069754,-0.000290,0.091081


In [383]:
ips = []
for i in city_ip_2.values():
    ips.append(i)

In [ ]:
ips

In [392]:
df_train['NEWCITY'][3]

'None'

In [393]:
for i in range(0,len(ips)):
    if ips[i] == 'nan':
        ips[i] = 'None'

In [394]:
le5 = preprocessing.LabelEncoder()
le5.fit_transform(ips)

array([3275,  132, 2781, ..., 4269, 3618, 2523], dtype=int64)

In [395]:
train_df_bert.insert(loc = 4,column='CITY',value = le5.transform(df_train['NEWCITY']))
test_df_bert.insert(loc = 4,column='CITY',value = le5.transform(df_test['NEWCITY']))


In [399]:
X_sm_4

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,CITY,0,1,2,3,4,...,374,375,376,377,378,379,380,381,382,383
0,0,2,3,2,3275,0.062621,0.030104,-0.049696,-0.002872,-0.077366,...,-0.005600,0.025201,0.041644,-0.008444,-0.049387,-0.011292,0.105604,-0.026650,0.009321,0.075258
1,0,2,3,2,132,0.052006,0.048003,0.000151,-0.057583,-0.041250,...,0.010245,0.078027,0.041478,0.027773,0.064026,-0.029505,0.008016,0.048607,0.014628,0.026060
2,0,2,3,2,2781,0.062621,0.030104,-0.049696,-0.002872,-0.077366,...,-0.005600,0.025201,0.041644,-0.008444,-0.049387,-0.011292,0.105604,-0.026650,0.009321,0.075258
3,0,3,3,13,2838,-0.033458,-0.012506,-0.067504,-0.047726,-0.135716,...,-0.086089,0.069374,0.112561,0.030620,-0.005797,-0.048223,-0.018869,-0.102295,0.032811,0.014253
4,1,7,3,6,1820,-0.039615,-0.005113,-0.023556,-0.009341,-0.089699,...,-0.000015,0.088376,0.020650,0.020895,0.016181,0.038231,-0.041902,-0.014496,0.114087,0.040825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159507,0,2,3,4,557,-0.000359,-0.024196,-0.014890,0.012445,-0.076092,...,-0.031090,0.020907,0.036616,0.016995,-0.028016,-0.057671,0.113683,-0.030909,0.083124,0.049123
159508,1,2,3,17,3453,0.045055,0.021149,0.004994,0.077177,0.024563,...,-0.045141,0.021548,0.031074,0.055836,0.022751,-0.107806,0.090147,0.008749,-0.021875,0.045993
159509,0,10,3,15,2601,-0.045443,0.055469,0.009946,-0.027206,-0.102328,...,-0.027863,0.028706,0.181448,0.057001,-0.090028,0.012535,0.169476,-0.048533,0.077124,0.028309
159510,0,7,3,6,1444,-0.005699,0.052850,-0.029038,-0.022706,-0.065682,...,-0.005977,0.102965,0.069319,0.033486,0.018661,-0.010073,0.091102,-0.008341,0.110261,-0.003359


In [400]:
rf_ft = RandomForestClassifier()
rf_ft.fit(X_sm_4,y_sm_4)
rf_pred = rf_ft.predict(test_df_bert)
test_test_df.IS_HCP = rf_pred
test_test_df.to_csv('ninetysix_test_csv.csv',index = False)

In [441]:
test_df_bert.drop(columns = ['CITY'],inplace = True)

In [443]:
test

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,0,1,2,3,4,5,...,374,375,376,377,378,379,380,381,382,383
0,0,2,3,2,0.062621,0.030104,-0.049696,-0.002872,-0.077366,-0.075697,...,-0.005600,0.025201,0.041644,-0.008444,-0.049387,-0.011292,0.105604,-0.026650,0.009321,0.075258
1,0,2,3,2,0.052006,0.048003,0.000151,-0.057583,-0.041250,-0.041908,...,0.010245,0.078027,0.041478,0.027773,0.064026,-0.029505,0.008016,0.048607,0.014628,0.026060
2,0,2,3,2,0.062621,0.030104,-0.049696,-0.002872,-0.077366,-0.075697,...,-0.005600,0.025201,0.041644,-0.008444,-0.049387,-0.011292,0.105604,-0.026650,0.009321,0.075258
3,0,3,3,13,-0.033458,-0.012506,-0.067504,-0.047726,-0.135716,-0.072514,...,-0.086089,0.069374,0.112561,0.030620,-0.005797,-0.048223,-0.018869,-0.102295,0.032811,0.014253
4,1,7,3,6,-0.039615,-0.005113,-0.023556,-0.009341,-0.089699,0.026324,...,-0.000015,0.088376,0.020650,0.020895,0.016181,0.038231,-0.041902,-0.014496,0.114087,0.040825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159507,1,9,3,0,0.015403,0.040283,0.083158,0.004275,-0.052587,-0.048977,...,0.015764,0.034877,0.013244,-0.027508,-0.016610,-0.006133,0.121018,-0.041594,0.038671,0.031398
159508,1,6,2,14,-0.024979,0.055374,0.054391,-0.023311,-0.018129,0.024034,...,-0.082210,0.058115,0.026155,-0.059786,0.047701,-0.033563,0.005062,-0.004509,0.030647,0.067393
159509,1,2,3,30,-0.010822,-0.012667,-0.064539,-0.001628,-0.089062,-0.035419,...,-0.015182,0.017011,0.062176,-0.001522,0.023632,-0.022312,0.056052,-0.065760,0.041970,0.036893
159510,2,6,2,14,-0.024979,0.055374,0.054391,-0.023311,-0.018129,0.024034,...,-0.082210,0.058115,0.026155,-0.059786,0.047701,-0.033563,0.005062,-0.004509,0.030647,0.067393


In [453]:
rf_ft_2 = RandomForestClassifier()
rf_ft_2.fit(X_sm_2,y_sm_2)
rf_pred = rf_ft_2.predict(test_df_pca)
test_test_df.IS_HCP = rf_pred
test_test_df.to_csv('ninetynine_test_csv.csv',index = False)

In [448]:
from sklearn.decomposition import PCA
pca = PCA()

In [450]:
X_sm_2 = pca.fit_transform(X_sm_2)
test_df_pca = pca.transform(test_df_bert)

In [451]:
X_sm_2

array([[-9.76946578e+00, -2.47129369e+00, -3.90026489e-01, ...,
         2.28146554e-10,  2.16508218e-15, -2.98137134e-16],
       [-9.77285976e+00, -2.46400116e+00, -3.85479040e-01, ...,
         1.81696687e-11, -7.03878743e-17,  1.60387594e-16],
       [-9.76946578e+00, -2.47129369e+00, -3.90026489e-01, ...,
         2.28146554e-10,  2.16508218e-15, -2.98137134e-16],
       ...,
       [ 1.81287238e+01, -5.06307758e-02,  5.45616610e-01, ...,
         1.21832317e-10,  3.02267378e-16, -4.90284553e-17],
       [ 1.84257370e+00,  2.54194084e+00,  1.76144713e+00, ...,
        -1.62648664e-10,  1.64033583e-15,  2.52675940e-16],
       [ 1.91258034e+01,  2.99534716e-02,  5.73613340e-01, ...,
        -1.47520587e-11,  3.06140174e-16, -1.26910497e-17]])

In [401]:
xgb_ft_5 = XGBClassifier()
xgb_ft_5.fit(X_sm_2,y_sm_2)
#xgb_ft_pred = xgb_ft_5.predict(test_df_bert)
#test_test_df.IS_HCP = xgb_ft_pred
#test_test_df.to_csv('ninetyeig_test_csv.csv',index = False)

## SBert embedding 768 dim

In [262]:
model_2 = SentenceTransformer('all-mpnet-base-v2')

In [267]:
embeddings_train_2 = []
for i in tqdm(df_train['SENTENCE']):
    embeddings_train_2.append(model_2.encode(i))

100%|██████████████████████████████████████████████████████████████████████████| 113936/113936 [32:29<00:00, 58.43it/s]


In [268]:
embeddings_test_2 = []
for i in tqdm(df_test['SENTENCE']):
    embeddings_test_2.append(model_2.encode(i))

100%|████████████████████████████████████████████████████████████████████████████| 28493/28493 [07:58<00:00, 59.58it/s]


In [269]:
df_bert_embeddings_train_2 = pd.DataFrame(embeddings_train_2)
df_bert_embeddings_test_2 = pd.DataFrame(embeddings_test_2)

In [272]:
train_df_bert_2 = pd.concat([train_df_cat,df_bert_embeddings_train_2],axis=1)
test_df_bert_2 = pd.concat([test_df_cat,df_bert_embeddings_test_2],axis=1)

In [273]:
train_df_bert_2

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,0,1,2,3,4,5,...,758,759,760,761,762,763,764,765,766,767
0,0,2,3,2,0.021360,0.066042,-0.004380,-0.050590,0.061386,0.023032,...,0.042208,-0.025322,-0.007173,-0.003284,-0.068691,-0.020589,0.011062,-0.015176,0.021282,-0.055205
1,0,2,3,2,0.067559,0.065148,0.018175,-0.023737,0.050214,0.017900,...,0.026749,-0.017365,0.030727,-0.019749,-0.070585,0.022996,0.006949,0.010079,0.000664,-0.054980
2,0,2,3,2,0.021360,0.066042,-0.004380,-0.050590,0.061386,0.023032,...,0.042208,-0.025322,-0.007173,-0.003284,-0.068691,-0.020589,0.011062,-0.015176,0.021282,-0.055205
3,0,3,3,13,0.073569,-0.002887,0.016271,-0.007649,0.003476,0.024253,...,-0.039788,0.027465,-0.043568,0.051637,-0.010747,0.034133,0.019535,0.017098,-0.012249,-0.040692
4,1,7,3,6,0.043774,0.027780,0.002219,-0.059022,0.042888,0.019702,...,-0.014710,-0.022304,-0.020554,0.020351,-0.057963,0.021488,0.004102,-0.024494,0.015830,-0.046659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113931,0,2,3,3,0.055944,0.064567,-0.001545,-0.069034,0.041144,0.024890,...,0.068009,-0.047787,-0.015318,0.000538,-0.065880,-0.002195,0.025192,-0.016397,0.056815,-0.042425
113932,0,2,3,3,0.020991,0.054183,-0.015175,-0.056778,0.047986,0.034257,...,0.031321,-0.026947,-0.000023,0.014797,-0.068249,-0.017128,0.014524,-0.007822,0.015995,-0.045908
113933,0,7,3,6,0.036140,0.044889,-0.006845,-0.062356,0.034694,0.032690,...,0.012868,-0.026135,-0.006624,0.018808,-0.064268,0.000628,-0.005253,-0.011942,0.044018,-0.059971
113934,1,2,3,7,0.030166,0.017365,0.013408,-0.011211,0.021141,0.013544,...,-0.009518,0.001215,0.014216,0.019487,-0.012434,-0.026632,0.009993,-0.025644,0.016007,-0.023149


In [275]:
train_df_bert_2.columns = train_df_bert_2.columns.astype('str')
test_df_bert_2.columns = test_df_bert_2.columns.astype('str')

In [276]:
smote = SMOTE(sampling_strategy=1.0)
X_sm_3,y_sm_3 = smote.fit_resample(train_df_bert_2,y)

y_sm_3.value_counts()

0    79756
1    79756
Name: IS_HCP, dtype: int64

In [454]:
X_sm_3

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,0,1,2,3,4,5,...,758,759,760,761,762,763,764,765,766,767
0,0,2,3,2,0.021360,0.066042,-0.004380,-0.050590,0.061386,0.023032,...,0.042208,-0.025322,-0.007173,-0.003284,-0.068691,-0.020589,0.011062,-0.015176,0.021282,-0.055205
1,0,2,3,2,0.067559,0.065148,0.018175,-0.023737,0.050214,0.017900,...,0.026749,-0.017365,0.030727,-0.019749,-0.070585,0.022996,0.006949,0.010079,0.000664,-0.054980
2,0,2,3,2,0.021360,0.066042,-0.004380,-0.050590,0.061386,0.023032,...,0.042208,-0.025322,-0.007173,-0.003284,-0.068691,-0.020589,0.011062,-0.015176,0.021282,-0.055205
3,0,3,3,13,0.073569,-0.002887,0.016271,-0.007649,0.003476,0.024253,...,-0.039788,0.027465,-0.043568,0.051637,-0.010747,0.034133,0.019535,0.017098,-0.012249,-0.040692
4,1,7,3,6,0.043774,0.027780,0.002219,-0.059022,0.042888,0.019702,...,-0.014710,-0.022304,-0.020554,0.020351,-0.057963,0.021488,0.004102,-0.024494,0.015830,-0.046659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159507,1,2,3,31,0.009184,0.015027,0.000718,-0.024812,0.035377,0.021854,...,-0.007614,-0.030701,-0.044373,0.041014,-0.011649,-0.037928,0.004819,-0.008040,-0.005929,-0.012161
159508,0,9,3,0,0.005813,0.001430,0.013353,-0.006059,0.013694,0.020630,...,-0.055149,-0.075448,-0.030487,0.076526,-0.033830,0.008324,-0.047949,-0.027348,0.036835,-0.009664
159509,1,2,3,8,0.064971,0.092458,-0.003138,-0.077862,0.017584,0.047669,...,0.018440,0.008377,-0.033171,0.002132,-0.025585,0.005525,0.009432,-0.039050,-0.015447,-0.010962
159510,1,2,3,30,0.036259,0.104709,0.004785,-0.060936,0.063851,0.030143,...,0.046163,-0.029323,-0.024914,0.035693,-0.064837,-0.010561,-0.005671,-0.055609,0.023045,-0.031896


In [455]:
rf_ft_2 = RandomForestClassifier()
rf_ft_2.fit(X_sm_3,y_sm_3)
rf_pred = rf_ft_2.predict(test_df_bert_2)
test_test_df.IS_HCP = rf_pred
test_test_df.to_csv('hundred_test_csv.csv',index = False)

In [279]:
vc_ft_soft_3.fit(X_sm_3,y_sm_3)
vc_pred_4 = vc_ft_soft_3.predict(test_df_bert_2)
test_test_df.IS_HCP = vc_pred_4
test_test_df.to_csv('ninetytwo_test_csv.csv',index = False)

In [280]:
rf_ft_2 = RandomForestClassifier()
rf_ft_2.fit(X_sm_3,y_sm_3)
rf_pred = rf_ft_2.predict(test_df_bert_2)
test_test_df.IS_HCP = rf_pred
test_test_df.to_csv('ninetythree_test_csv.csv',index = False)

In [282]:
xgb_ft_5 = XGBClassifier()
xgb_ft_5.fit(X_sm_3,y_sm_3)
xgb_ft_pred = xgb_ft_5.predict(test_df_bert_2)
test_test_df.IS_HCP = xgb_ft_pred
test_test_df.to_csv('ninetyfour_test_csv.csv',index = False)

In [ ]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

space={ 'objective':'binary:logistic',
        'learning_rate':hp.uniform('learning_rate',0.1,0.4),
        'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 0,9),
        'reg_alpha' : hp.quniform('reg_alpha', 0,1,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_sm_3, y_sm_3, test_size=0.20, random_state=42)
def objective(space):
    clf=XGBClassifier(objective = space['objective'],learning_rate = space['learning_rate'],
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train_2, y_train_2), ( X_test_2, y_test_2)]
    
    clf.fit(X_train_2, y_train_2,
             eval_set=evaluation,eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    
    pred = clf.predict(X_test_2)
    accuracy = accuracy_score(y_test_2, pred)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

trials = Trials()

best_hyperparams_2 = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)
best_hyperparams_3['max_depth'] = int(best_hyperparams['max_depth'])
print(best_hyperparams_3)

In [298]:
best_hyperparams_2['max_depth'] = int(best_hyperparams['max_depth'])
xgb_ft_5 = XGBClassifier()
xgb_ft_5.set_params(**best_hyperparams_2)
xgb_ft_5.fit(X_sm_3,y_sm_3)
xgb_ft_pred = xgb_ft_5.predict(test_df_bert_2)
test_test_df.IS_HCP = xgb_ft_pred
test_test_df.to_csv('ninetyfive_test_csv.csv',index = False)

## IP

In [303]:
temp_df = pd.read_csv('train_modified.csv')
temp_df_test = pd.read_csv('test_modified.csv')

In [304]:
city_ip = {}
for i,j in zip(temp_df['BIDREQUESTIP'],temp_df['USERCITY']):
    city_ip[i] = j
for i,j in zip(temp_df_test['BIDREQUESTIP'],temp_df_test['USERCITY']):
    city_ip[i] = j
    

In [305]:
count = 0
for i,j in city_ip.items():
    if type(j) == float:
        count+=1
count

2126

In [310]:
miss_ip = {}
for i,j in city_ip.items():
    if str(j) == "nan":
        miss_ip[i] = None

In [ ]:
miss_ip

In [325]:
from ipdata import ipdata
from pprint import pprint
ipd=ipdata.IPData('b6418bb1929a578f5b509bc7132368e141084ea97d7b97cdac15cf02')
def look(ipadd):
    res=ipd.lookup(ipadd)
    return(res['region'])

In [326]:
for i,j in enumerate(miss_ip.keys()):
    if i == 1499:
        print('reached')
        break
    else:
        miss_ip[j] = look(j)

reached


In [329]:
len(miss_list)

627

In [330]:
for i in tqdm(miss_list):
    miss_ip[i] = look(i)

100%|████████████████████████████████████████████████████████████████████████████████| 627/627 [00:35<00:00, 17.84it/s]


In [331]:
def iptocity(ip_add):
    if (city_ip[ip_add] == None) and (miss_ip[i] == None):
        return None
    elif (city_ip[ip_add]!= None):
        return city_ip[ip_add]
    else:
        return miss_ip[ip_add]

df_train['NEWCITY'] = df_train['BIDREQUESTIP'].apply(iptocity)
df_test['NEWCITY'] = df_test['BIDREQUESTIP'].apply(iptocity)
        
    

In [343]:
temp_df ['USERCITY'] = temp_df['USERCITY'].astype('str')

In [357]:
city_ip_2 = {}
for i,j in zip(temp_df['BIDREQUESTIP'],temp_df['USERCITY']):
    city_ip_2[i] = str(j)
for i,j in zip(temp_df_test['BIDREQUESTIP'],temp_df_test['USERCITY']):
    city_ip_2[i] = str(j)

In [365]:
df_train.NEWCITY = df_train['NEWCITY'].fillna(value = 'None')

In [374]:
df_test.NEWCITY = df_test['NEWCITY'].fillna(value = 'None')

In [367]:
df_train['NEWCITY']

0             Portland
1            Arlington
2          New Meadows
3                 None
4              Houston
              ...     
113931    Philadelphia
113932        Van Nuys
113933            None
113934      Wilmington
113935    White Plains
Name: NEWCITY, Length: 113936, dtype: object

In [356]:
temp_df ['USERCITY'][3]

'nan'

In [359]:
count = 0
for i,j in city_ip_2.items():
    if str(j) == 'nan':
        count+=1
count

2126

In [360]:
df_train['NEWCITY'].isna().sum()

6352

In [403]:
train_df_taxonomy = train_df_bert.copy()

In [405]:
train_df_taxonomy['IS_HCP'] = y

In [417]:
y_taxonomy = temp_df[temp_df['IS_HCP']==1.0]['TAXONOMY']

In [414]:
train_df_taxonomy = train_df_taxonomy[train_df_taxonomy['IS_HCP']==1.0]

In [419]:
le6 =preprocessing.LabelEncoder()
le6.fit_transform(y_taxonomy)
y_taxonomy = le6.transform(y_taxonomy)

In [421]:
X_train_tax, X_test_tax, y_train_tax, y_test_tax = train_test_split(train_df_taxonomy, y_taxonomy, test_size=0.33, random_state=42)

In [ ]:
rf_ft_tax = RandomForestClassifier()
rf_ft_tax.fit(X_train_tax,y_train_tax)
rf_ft_tax_pred = rf_ft_tax.predict(X_test_tax)
print(classification_report(y_test_tax,rf_ft_tax_pred))

In [426]:
from sklearn.feature_selection import RFECV

In [432]:
df_train.SENTENCE

,ID,IS_HCP
0,115501,0
1,115502,1
2,115503,0
3,115504,0
4,115505,1
...,...,...
28488,143989,0
28489,143990,0
28490,143991,0
28491,143992,0


In [437]:
df_train.to_csv('train_modifed_3.csv',columns =['SENTENCE','IS_HCP','TAXONOMY'],index = False)
df_test.to_csv('test_modifed_3.csv',columns =['ID','SENTENCE'],index = False)


In [ ]:
model_CBC_2 = ctb.CatBoostClassifier(iterations = 200)
model_CBC_2.fit(X_train_tax, y_train_tax)
ct_tax_pred = model_CBC_2.predict(X_test_tax)
print(classification_report(y_test_tax,ct_tax_pred))

In [412]:
(temp_df['IS_HCP']==1.0).sum()

34180

In [413]:
temp_df['IS_HCP'].value_counts()

0    79756
1    34180
Name: IS_HCP, dtype: int64

## Rest

In [139]:
df_train.BIDREQUESTIP.unique().shape

(33664,)

In [104]:
df_train[df_train['USERCITY']==np.nan]['BIDREQUESTIP']

Series([], Name: BIDREQUESTIP, dtype: object)

In [122]:
user_city = df_train['USERCITY']
url_city = user_city.replace(np.nan,0, regex=True)

In [119]:
import math

In [125]:
miss_list = []
for i,j in zip(url_city,df_train['BIDREQUESTIP']):
    if str(i) == "0":
        miss_list.append(j)

In [127]:
len(miss_list)

6359

In [129]:
uni_ip_miss = pd.Series(miss_list).unique()

In [130]:
len(uni_ip_miss)

1948

In [134]:
ip = {}
for i,j in zip(df_train['USERCITY'],df_train['BIDREQUESTIP']):
    if ip.get(i,"unknown") == "unknown":
        ip[j]=i

In [170]:
ip['66.249.66.73']

nan

In [135]:
len(ip.keys())

33664

In [172]:
missing = []
for k,v in ip.items():
    if type(v) == float:
        missing.append(k)
len(missing)

1896

In [173]:
missing_1 = missing[:1500]
missing_2 =  missing[1500:]

In [176]:
len(missing_2)

396

In [195]:
counter = 0
for i in ip.keys():
    if type(ip[i]) == float:
        counter+=1
counter

0

In [193]:
from ipdata import ipdata
from pprint import pprint
ipd=ipdata.IPData('ac1c77f2f0547302145150b005b2edfa1b4b6a7cfcbea203e0a867f6')
def look(ipadd):
    res=ipd.lookup(ipadd)
    return(res['region'])

In [190]:
for i in missing_1:
    ip[i]= str(look(i))

In [194]:
for i in missing_2:
    ip[i]= str(look(i))

In [197]:
def ip_lookup(ip_add):
    return ip[ip_add]

df_train['USERCITY'] = df_train['BIDREQUESTIP'].apply(lambda x : ip_lookup(x))

In [199]:
df_train['USERCITY'].value_counts()

New York         5184
Brooklyn         3221
None             2563
St Louis         2546
Los Angeles      2156
                 ... 
Lapoint             1
Redwood Falls       1
Halstead            1
Benwood             1
Gambier             1
Name: USERCITY, Length: 4451, dtype: int64

In [200]:
df_test['NEW_CITY'] = df_test['BIDREQUESTIP'].apply(lambda x : ip_lookup(x))

KeyError: '98.213.215.3'

## Taxonomy

In [504]:
train_df_taxonomy.drop(columns = ['CITY'],inplace = True)

C:\Users\USER\AppData\Local\Temp\ipykernel_12840\2141750290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_taxonomy.drop(columns = ['CITY'],inplace = True)


In [505]:
train_df_taxonomy

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,0,1,2,3,4,5,...,375,376,377,378,379,380,381,382,383,IS_HCP
3,0,3,3,13,-0.033458,-0.012506,-0.067504,-0.047726,-0.135716,-0.072514,...,0.069374,0.112561,0.030620,-0.005797,-0.048223,-0.018869,-0.102295,0.032811,0.014253,1
8,1,2,3,2,0.035835,0.056559,0.005856,0.063137,0.019431,-0.029709,...,0.035598,0.011175,0.042429,0.017862,-0.057412,0.084522,0.046900,-0.018370,-0.000325,1
21,0,3,3,13,0.057392,0.022769,-0.048059,0.006806,-0.106488,-0.019674,...,0.106294,0.014401,0.009704,-0.043004,-0.164117,0.050447,-0.055480,0.032829,-0.056874,1
29,0,7,3,6,-0.005699,0.052850,-0.029038,-0.022706,-0.065682,-0.025156,...,0.102965,0.069319,0.033486,0.018661,-0.010073,0.091102,-0.008341,0.110261,-0.003359,1
35,0,2,3,31,-0.064100,0.054472,0.016383,0.010234,-0.060284,0.004476,...,0.014604,0.035677,0.044102,0.000944,-0.064837,-0.058292,0.010316,0.051608,0.081141,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113931,0,2,3,3,0.010681,0.081000,-0.066219,0.003094,-0.111840,-0.052544,...,0.058592,0.054049,0.052852,-0.012197,-0.059723,0.042657,-0.026619,0.041272,0.014037,1
113932,0,2,3,3,0.008793,0.039040,-0.004619,-0.010275,-0.085800,-0.061926,...,0.095038,0.007995,0.045544,0.001883,-0.017897,0.073166,-0.012864,0.019074,0.058617,1
113933,0,7,3,6,-0.005699,0.052850,-0.029038,-0.022706,-0.065682,-0.025156,...,0.102965,0.069319,0.033486,0.018661,-0.010073,0.091102,-0.008341,0.110261,-0.003359,1
113934,1,2,3,7,0.003200,0.047974,0.016732,0.045390,0.001444,0.030114,...,0.104969,0.089438,-0.008973,0.005175,-0.060500,0.076859,-0.069754,-0.000290,0.091081,1


In [512]:
y_taxonomy = df_train[df_train['IS_HCP']==1.0]['TAXONOMY']
y_taxonomy = y_taxonomy.fillna(value = 'MISSING')
y_taxonomy.nunique()

208

In [514]:
le6 = preprocessing.LabelEncoder()
y_taxonomy = le6.fit_transform(y_taxonomy)
len(y_taxonomy)

34180

In [506]:
test_test_df = pd.read_csv('eightysix_test_csv.csv')

In [508]:
test_df_bert.insert(loc = 4,column='IS_HCP',value = test_test_df['IS_HCP'])

In [517]:
test_df_bert_tax = test_df_bert[test_df_bert['IS_HCP']==1.0]
test_df_bert_tax.drop(columns = ['IS_HCP'],inplace = True)

C:\Users\USER\AppData\Local\Temp\ipykernel_12840\2722392768.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df_bert_tax.drop(columns = ['IS_HCP'],inplace = True)


In [518]:
test_df_bert_tax

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,0,1,2,3,4,5,...,374,375,376,377,378,379,380,381,382,383
1,1,2,3,22,0.094528,-0.016281,0.024730,0.011690,-0.147179,-0.118570,...,-0.127606,0.051345,0.091650,0.006316,-0.051278,-0.027012,0.098344,0.055415,0.043057,0.024547
4,0,2,3,30,-0.025487,0.062336,0.020333,0.031348,-0.049694,-0.082555,...,-0.001440,0.039353,0.012410,-0.047881,0.078327,-0.007633,0.131235,-0.102498,0.019462,0.089912
10,0,10,3,15,-0.011279,0.067786,0.026031,-0.047196,-0.153808,-0.031492,...,-0.019465,0.059365,0.165211,0.041470,-0.031283,0.067625,0.053551,-0.064811,0.110050,0.051796
12,0,9,3,0,0.029113,0.087491,0.054646,0.081216,-0.071921,-0.035567,...,-0.050208,0.127540,0.002006,0.048931,0.010584,-0.037779,0.056120,-0.036780,0.019623,0.085504
29,1,2,3,3,-0.005615,0.052251,0.011641,-0.005719,-0.114535,-0.055044,...,-0.029370,0.087737,0.035593,0.073274,-0.047362,-0.025663,0.055252,0.044697,0.014369,0.064957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28470,1,2,3,7,-0.061752,0.016460,-0.045373,-0.026560,-0.082764,0.027928,...,-0.051697,0.012193,0.034566,-0.023069,0.023961,-0.040771,-0.011552,-0.027522,0.065873,0.126319
28472,1,2,3,3,0.072370,0.090226,0.040983,0.028344,0.013454,0.045222,...,-0.020843,0.049434,0.001438,0.058054,-0.005235,-0.055423,0.052438,0.028210,-0.002161,0.047997
28474,0,2,3,30,-0.010822,-0.012667,-0.064539,-0.001628,-0.089062,-0.035419,...,-0.015182,0.017011,0.062176,-0.001522,0.023632,-0.022312,0.056052,-0.065760,0.041970,0.036893
28476,0,2,3,29,-0.007845,0.021832,-0.063514,0.008187,0.020227,-0.040161,...,-0.028450,0.057766,0.060384,0.049326,-0.012869,-0.042326,0.093525,-0.021216,0.038002,0.041082


In [520]:
 class_dist = pd.Series(y_taxonomy).value_counts()

In [521]:
low_sample_classes = [c for c in range(208) if sum(y_taxonomy == c) < 100]

In [522]:
len(low_sample_classes)

176

In [529]:
low_samp = {}
for j , k in zip(class_dist.index,class_dist):
    if k <100:
        low_samp[j]=100

In [ ]:
low_samp

In [524]:
len(low_samp)

176

In [528]:
y_taxonomy

array([149, 151, 143, ..., 112, 149, 149])

In [525]:
from imblearn.over_sampling import RandomOverSampler

In [531]:
ros = RandomOverSampler(sampling_strategy=low_samp)
X_train_resampled, y_train_resampled = ros.fit_resample(train_df_taxonomy, y_taxonomy)


In [533]:
pd.Series(y_train_resampled).value_counts()

149    7930
143    6621
43     3997
50     3190
207    1867
       ... 
27      100
45      100
28      100
15      100
95      100
Length: 208, dtype: int64

In [535]:
len(y_train_resampled)

49034

In [542]:
X_train_resampled.drop(columns = ['IS_HCP'],inplace = True)

In [538]:
X_test_tax.drop(columns = ['CITY'],inplace = True)

In [539]:
X_test_tax

,DEVICETYPE,PLATFORM_ID,PLATFORMTYPE,WEBSITE,0,1,2,3,4,5,...,375,376,377,378,379,380,381,382,383,IS_HCP
23516,0,2,3,8,-0.028396,0.048858,-0.013073,0.025552,-0.033497,-0.032372,...,0.024263,-0.008740,0.035946,-0.033504,-0.057558,0.093570,-0.031515,0.046827,0.081864,1
63133,0,2,3,19,-0.000970,0.034819,-0.069188,0.047815,-0.053418,-0.053378,...,-0.010813,0.041591,-0.023076,0.016036,-0.014039,0.084938,-0.008320,0.050946,0.093299,1
84865,0,6,2,14,0.008684,-0.061361,0.007171,-0.014539,0.041258,0.007897,...,0.017935,-0.054351,0.016503,0.003400,-0.068768,-0.036647,-0.019348,0.008631,0.062533,1
36675,0,2,3,21,0.018542,-0.030602,-0.084834,0.031817,-0.031580,-0.015734,...,0.052752,0.066440,-0.001503,-0.002875,-0.027536,0.004372,-0.014765,0.065073,-0.000750,1
99077,0,2,3,31,-0.026119,-0.033836,-0.004597,0.007481,-0.009047,0.012391,...,0.069716,0.026685,0.001886,0.002114,-0.041791,0.105231,-0.036691,-0.024000,0.066349,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101379,0,2,3,20,0.010538,0.008147,-0.012990,0.045029,-0.042845,0.021248,...,0.019625,0.022425,0.047462,-0.011184,-0.123885,0.029340,-0.018087,-0.026642,0.042917,1
57070,1,2,3,29,-0.020976,0.019264,-0.067132,-0.015295,0.005159,-0.006460,...,0.033230,0.032550,-0.004199,0.038387,0.019552,0.037726,-0.029598,0.010277,0.020238,1
39291,0,2,3,30,-0.011524,0.055273,-0.007703,0.030232,-0.049911,-0.049180,...,0.021341,0.028424,0.016515,-0.001598,0.019610,0.072725,0.006297,-0.015154,0.016640,1
4872,0,2,3,19,-0.000970,0.034819,-0.069188,0.047815,-0.053418,-0.053378,...,-0.010813,0.041591,-0.023076,0.016036,-0.014039,0.084938,-0.008320,0.050946,0.093299,1


In [544]:
rf_ft_2 = RandomForestClassifier()
rf_ft_2.fit(X_train_resampled,y_train_resampled)

RandomForestClassifier()

In [545]:
rf_pred = rf_ft_2.predict(test_df_bert_tax)

In [546]:
rf_pred = le6.inverse_transform(rf_pred)

In [553]:
rf_pred

array(['2084P0800X', '2084N0400X', '363LF0000X', ..., '2084P0800X',
       '2084N0400X', '207U00000X'], dtype=object)

In [549]:
test_df_bert_tax.index

Int64Index([    1,     4,    10,    12,    29,    31,    35,    36,    37,
               41,
            ...
            28457, 28458, 28462, 28463, 28468, 28470, 28472, 28474, 28476,
            28480],
           dtype='int64', length=8479)

In [550]:
test_test_df

,ID,IS_HCP
0,115501,0
1,115502,1
2,115503,0
3,115504,0
4,115505,1
...,...,...
28488,143989,0
28489,143990,0
28490,143991,0
28491,143992,0


In [551]:
test_test_df.insert(1,'TAXONOMY','')

In [552]:
test_test_df

,ID,TAXONOMY,IS_HCP
0,115501,,0
1,115502,,1
2,115503,,0
3,115504,,0
4,115505,,1
...,...,...,...
28488,143989,,0
28489,143990,,0
28490,143991,,0
28491,143992,,0


In [554]:
for i,v in zip(test_df_bert_tax.index,rf_pred):
    if v != 'MISSING':
        test_test_df.loc[i, 'TAXONOMY'] = v


In [556]:
test_test_df.to_csv('taxonomy.csv',index = False)